In [2]:
import sys
sys.path.append('../build')
import IPSModule as ips

In [ ]:
import numpy as np

num_particles = 10
p = ips.Particles(num_particles)

# use random seed, to generate random numbers
pmin, pmax = -1, 1
vmin, vmax = -2, 2

rad = 2.0

np.random.seed(42)  # for reproducibility

for i in range(num_particles):
    for d in range(2):
        p.get_positions()[d][i] = np.random.uniform(pmin, pmax)
        p.get_velocities()[d][i] = np.random.uniform(vmin, vmax)

pair_force_config = {
    "type": "Spring",
    "k": 1.0,
    "r_0": 1.0
}

confinement_config = {
    "type": "Radial",
    "rad": rad
}
simulator = ips.IPS_Simulator(p)
simulator.init(pair_force_config, confinement_config)

In [4]:

def kinetic_energy(velocity):
    return 0.5 * np.linalg.norm(velocity) ** 2

def potential_energy(position):
    total_potential_energy = 0
    for i in range(num_particles):
        for j in range(i + 1, num_particles):
            pos_i = np.array([position[0][i], position[1][i]])
            pos_j = np.array([position[0][j], position[1][j]])
            dist = np.linalg.norm(pos_i - pos_j)
            total_potential_energy += 0.5 * (dist - 1) ** 2
    return total_potential_energy

def calculate_total_energy(positions, velocities):
    total_kinetic_energy = 0
    total_potential_energy = 0
    for i in range(num_particles):
        vel = np.array([velocities[0][i], velocities[1][i]])
        total_kinetic_energy += kinetic_energy(vel)
    total_potential_energy = potential_energy(positions)
    return total_kinetic_energy + total_potential_energy

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.display import HTML, display
from matplotlib.gridspec import GridSpec

num_steps = 10000
draw_interval = 100
dt = 0.001

fig = plt.figure(figsize=(12, 6))
gs = GridSpec(1, 2, width_ratios=[1, 1])

ax1 = fig.add_subplot(gs[0])
scat = ax1.scatter([], [], s=50)

ax2 = fig.add_subplot(gs[1])
total_energy_line, = ax2.plot([], [], label='Total Energy')
kinetic_energy_line, = ax2.plot([], [], label='Kinetic Energy')
potential_energy_line, = ax2.plot([], [], label='Potential Energy')

w_min, w_max = -4, 4

initial_energy = calculate_total_energy(p.get_positions(), p.get_velocities())

def init():
    circle1 = plt.Circle((0, 0), rad, color='r', fill=False)
    circle2 = plt.Circle((0, 0), rad + 1, color='r', fill=False)
    ax1.add_artist(circle1)
    ax1.add_artist(circle2)
    ax1.set_xlim(w_min, w_max)
    ax1.set_ylim(w_min, w_max)
    ax1.set_aspect('equal')
    ax1.set_title('Particle Positions')
    ax2.set_xlim(0, num_steps // draw_interval)
    expected_energy = num_particles * vmax ** 2
    factor = 0.5
    ax2.set_ylim(0, initial_energy + factor * initial_energy)
    ax2.set_title('Energy Over Time')
    ax2.set_xlabel('Time Step')
    ax2.set_ylabel('Energy')
    ax2.legend()
    return scat, total_energy_line, kinetic_energy_line, potential_energy_line

total_energies = []
kinetic_energies = []
potential_energies = []

def update(frame):
    simulator.integrate_n_steps(dt, draw_interval)

    positions = p.get_positions()
    velocities = p.get_velocities()
    total_energy = calculate_total_energy(positions, velocities)
    kinetic_energy_val = sum(kinetic_energy(np.array([velocities[0][i], velocities[1][i]])) for i in range(num_particles))
    potential_energy_val = potential_energy(positions)
    
    total_energies.append(total_energy.item())
    kinetic_energies.append(kinetic_energy_val)
    potential_energies.append(potential_energy_val)
    
    offsets = np.c_[positions[0], positions[1]]
    colors = ['red'] + ['blue'] * (num_particles - 1)  # Set the first particle's color to red
    scat.set_offsets(offsets)
    scat.set_color(colors)
    
    total_energy_line.set_data(range(len(total_energies)), total_energies)
    kinetic_energy_line.set_data(range(len(kinetic_energies)), kinetic_energies)
    potential_energy_line.set_data(range(len(potential_energies)), potential_energies)
    
    return scat, total_energy_line, kinetic_energy_line, potential_energy_line

ani = animation.FuncAnimation(fig, update, frames=range(0, num_steps, draw_interval), init_func=init, blit=True)
json = ani.to_jshtml()
plt.close()
display(HTML(json))

In [ ]:
import time
num_particles_list = [10, 50, 100, 200, 500, 1000, 1500]
elapsed_times = []

for num_particles in num_particles_list:
    p = ips.Particles(num_particles)

    # use random seed, to generate random numbers
    pmin, pmax = -1, 1
    vmin, vmax = -2, 2

    np.random.seed(42)  # for reproducibility

    for i in range(num_particles):
        for d in range(2):
            p.get_positions()[d][i] = np.random.uniform(pmin, pmax)
            p.get_velocities()[d][i] = np.random.uniform(vmin, vmax)

    simulator = ips.IPS_Simulator(p)

    start = time.time()
    simulator.integrate_n_steps(0.0001, 200)
    end = time.time()
    elapsed_time = end - start
    elapsed_times.append(elapsed_time)
    print(f"Elapsed time for {num_particles} particles: {elapsed_time}")

In [ ]:
# Plot the results
plt.figure()
plt.plot(num_particles_list, elapsed_times, marker='o')
plt.xlabel('Number of Particles')
plt.ylabel('Elapsed Time (s)')
plt.title('Simulation Time vs Number of Particles')
plt.grid(True)
plt.show()